In [58]:
import pandas as pd
import numpy as np
import scipy as sp
# Open graphs in new cells in the page rather than in a separate window
%matplotlib inline 
import matplotlib.pyplot as plt 
plt.rcParams["figure.figsize"] = (15, 5)  # set a default figure size
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

import time
import datetime

In [6]:
inputFile = pd.read_csv('answers.csv')

#df = inputFile[:-100]
df = inputFile[:20000]

testdf = inputFile[-100:]

In [7]:
print(len(df))

20000


In [10]:
# Look at the first 3 rows
df[:3]

,Unnamed: 0,qid,i,qs,qt,tags,qvc,qac,aid,j,as,at
0,1,563355,62701.0,0,1235000081,"php,error,gd,image-processing",220,2,563372,67183.0,2,1235000501
1,2,563355,62701.0,0,1235000081,"php,error,gd,image-processing",220,2,563374,66554.0,0,1235000551
2,3,563356,15842.0,10,1235000140,"lisp,scheme,subjective,clojure",1047,16,563358,15842.0,3,1235000177


Columns values,
qid: Unique question id

i: User id of questioner

qs: Score of the question

qt: Time of the question (in epoch time)

tags: a comma-separated list of the tags associated with the question. Examples of tags are ``html'', ``R'', ``mysql'', ``python'', and so on; often between two and six tags are used on each question.

qvc: Number of views of this question (at the time of the datadump)

qac: Number of answers for this question (at the time of the datadump)

aid: Unique answer id

j: User id of answerer

as: Score of the answer

at: Time of the answer

In [11]:
df['tags'][:10]

0     php,error,gd,image-processing
1     php,error,gd,image-processing
2    lisp,scheme,subjective,clojure
3    lisp,scheme,subjective,clojure
4    lisp,scheme,subjective,clojure
5    lisp,scheme,subjective,clojure
6    lisp,scheme,subjective,clojure
7    lisp,scheme,subjective,clojure
8    lisp,scheme,subjective,clojure
9    lisp,scheme,subjective,clojure
Name: tags, dtype: object

In [12]:
features =  set()

def feature_selection(data):
    for rows in data:
        for feat in  rows.split(','):
            features.add(feat)
            
            
                        

In [13]:
feature_selection(df['tags']) # Get list of all the unique tags in features
print(len(features))

3721


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(norm='l2', vocabulary=list(features))

In [15]:
training_matrix =  tf.fit_transform(df['tags'])
training_feature_names = tf.get_feature_names() 

In [16]:
test_matrix =  tf.fit_transform(testdf['tags'])


In [17]:
#print(training_feature_names)

In [18]:
training_matrix[:2]

<2x3721 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [19]:
X=training_matrix
y = test_matrix

In [20]:
kmeans = KMeans(n_clusters=20, random_state=0).fit(X)

In [21]:
predication = []

predication = kmeans.predict(y)

len(predication)
print(predication)

[ 2  3  3  3  3  3  3  3  3  2  3  3  3  3 10 10 10 10  3  3  3 12 12 12 12
 19 19  5  1  3  2 19 10 10 11 11 11 11 11 11  3  7  7  3  3  3  3  3  1 12
 12 12 12 18 18 18  3  3  3  1 12 12 12 12 11 11 11 11 11 11 11  3  3  3  3
  3  3  3  3  3 19 19 11 11 11 11  3  3  3  3  3  3  3  3  3 14 14 16 16 18]


In [33]:
print(kmeans.cluster_centers_[19])

[ 0.  0.  0. ...,  0.  0.  0.]


In [34]:
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, X)


In [35]:
##Closest node from the centroids
print(closest)  

[ 905  905  905  905 8411 5992  905  905 5011 3376  147  905  905  905  905
  905  905  418  376  905]


In [51]:
EstimatedTimes = []
#print(type(df))

for closenode in closest:
    #print (df.at[closenode, 'qt'])
    #print (df.at[closenode, 'at'])
    answerTime = (df.at[closenode, 'at']) - (df.at[closenode, 'qt'])
    EstimatedTimes.insert(answerTime)    

AttributeError: 'list' object has no attribute 'add'

In [ ]:
print(EstimatedTimes)

In [53]:
predications= []
predications= predication[:100]

print(predications)

[ 2  3  3  3  3  3  3  3  3  2  3  3  3  3 10 10 10 10  3  3  3 12 12 12 12
 19 19  5  1  3  2 19 10 10 11 11 11 11 11 11  3  7  7  3  3  3  3  3  1 12
 12 12 12 18 18 18  3  3  3  1 12 12 12 12 11 11 11 11 11 11 11  3  3  3  3
  3  3  3  3  3 19 19 11 11 11 11  3  3  3  3  3  3  3  3  3 14 14 16 16 18]


In [75]:
PredicationTime = []
for predication in predications:
    closestNode = (closest[predication])
    answerTime = (df.at[closestNode, 'at']) - (df.at[closestNode, 'qt'])
    #print(answerTime)
    #print(datetime.datetime.fromtimestamp(answerTime).strftime('%c'))
    PredicationTime.append(time.strftime('%M:%S', time.localtime(answerTime)))  #Fix this part

In [76]:
print(PredicationTime)

['05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '04:33', '04:33', '04:33', '04:33', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '09:11', '05:16', '05:16', '05:16', '05:16', '04:33', '04:33', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:55', '05:55', '05:55', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:16', '05:55']


In [77]:
f = open('Results/result.dat', 'w')

count = 0
for predictionValue in PredicationTime:
    #print(predictionValue)
    f.write(str(predictionValue)+'\n')
    count+=1
print("count : ",count)
print("--The End--")

count :  100
--The End--
